In [480]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [481]:
bikesdata = pd.read_csv('../input/bike-sharing-data-set/day.csv')
bikesdata.head()

In [482]:
bikesdata.shape

In [483]:
bikesdata.describe()

In [484]:
bikesdata.info()

## Cleanup unnecessary columns

In [485]:
# Check all the values in yr column to see if it can be dropped
bikesdata.yr.unique()

In [486]:
# instant is unique for each row
print(len(bikesdata.instant.unique()))

# dteday is unique for each row
print(len(bikesdata.dteday.unique()))

In [487]:
# Drop instant and dteday columns since they are all unique values
bikesdata.drop(['instant', 'dteday'], axis = 1, inplace=True)

# Drop registered and casual columns since we are only interested in cnt and not casual, registered (casual + registered = cnt)
bikesdata.drop(['casual', 'registered'], axis=1,inplace=True)


In [488]:

bikesdata.head()

## Visualizing the data
- Check for any collinearity


In [489]:
import matplotlib.pyplot as plt
import seaborn as sns

In [490]:
# Visualize the pairwise relationship between target and predictors
sns.pairplot(bikesdata)

# Observations from above pair plot
 
- season, yr, mnth, holiday, weekday, workingday, weathersit are categorical columns
- there seems be a linear relationship (highly corelated)  between atemp and temp columns and we can drop one of these columns

# Visualising Categorical Variables
Making boxplot of categorical columns (season, yr, mnth, holiday, weekday, workingday, weathersit)

In [491]:
plt.figure(figsize=(20,10))
plt.subplot(3,3,1)
sns.boxplot(x='season', y='cnt', data=bikesdata)
plt.subplot(3,3,2)
sns.boxplot(x='yr', y='cnt', data=bikesdata)
plt.subplot(3,3,3)
sns.boxplot(x='mnth', y='cnt', data=bikesdata)
plt.subplot(3,3,4)
sns.boxplot(x='holiday', y='cnt', data=bikesdata)
plt.subplot(3,3,5)
sns.boxplot(x='weekday', y='cnt', data=bikesdata)
plt.subplot(3,3,6)
sns.boxplot(x='workingday', y='cnt', data=bikesdata)
plt.subplot(3,3,7)
sns.boxplot(x='weathersit', y='cnt', data=bikesdata)


### Observations from above boxplots
- Spring(1) has less sales than other seasons
- 2019 has higher sales than 2018, but also has some outliers
- Months 4-7 have higher sales, which may be reflective of the summer weather
- Day of week, working day (neither weekend nor holiday) and holiday doesn't seem to have much effect on the sales
- Since holiday is subset of workingday (but negation) and doesn't seem to have much effect on sales, may be ideal to drop one or both the columns
- Weathersit shows better weather (1,2) clearly has better sales than bad weather (3,4)


## Data Prepepration
- Convert categorical columns to dummy variables

In [492]:
bikesdata.head(10)

In [493]:
# List of categorical columns

category_col =  ['season', 'mnth', 'weekday', 'weathersit']

seasons = pd.get_dummies(bikesdata['season'], prefix='season', drop_first=True)
seasons.rename(columns={'season_2': 'season_summer', 'season_3': 'season_fall', 'season_4': 'season_winter' }, inplace=True)

mnths = pd.get_dummies(bikesdata['mnth'], prefix='month', drop_first=True);
mnths.rename(columns={
        'month_2': 'mnth_feb',
        'month_3': 'mnth_mar',
        'month_4': 'mnth_apr',
        'month_5': 'mnth_may',
        'month_6': 'mnth_jun',
        'month_7': 'mnth_jul',
        'month_8': 'mnth_aug',
        'month_9': 'mnth_sep',
        'month_10': 'mnth_oct',
        'month_11': 'mnth_nov',
        'month_12': 'mnth_dec'
    }, inplace=True)

weekdays = pd.get_dummies(bikesdata['weekday'],prefix='dayofweek', drop_first=True)

weekdays.rename(columns={
    'dayofweek_1': 'day_tue',
    'dayofweek_2': 'day_wed',
    'dayofweek_3': 'day_thu',
    'dayofweek_4': 'day_fri',
    'dayofweek_5': 'day_sat',
    'dayofweek_6': 'day_sun'
}, inplace=True)

weathersits = pd.get_dummies(bikesdata['weathersit'], prefix='weather', drop_first=True)
weathersits.rename(columns={
    'weather_2': 'wthr_cloudy_misty',
    'weather_3': 'wthr_snow_storms'
}, inplace=True)

bikesdata = pd.concat([bikesdata, seasons, mnths, weekdays, weathersits], axis=1)


In [494]:
#Drop the original categories columns which are encoded
bikesdata.drop(['season', 'mnth', 'weekday', 'weathersit'], axis=1, inplace=True)

## Outlier Detection and Treatment

In [495]:
def outlier_treatment(df,column=None):
  """This function accepts a dataframe and a numerical column name for which outliers are treated"""
  df = df[df[column] < df[column].quantile(0.99)]
  return df

In [496]:
def outlier_treatment_columns(df,columns):
    for column in columns:
        df = df[df[column] < df[column].quantile(0.99)]
    return df

In [497]:
num_vars = ['temp', 'atemp', 'hum', 'windspeed']

In [498]:
plt.figure(figsize=(20,10))
plt.subplot(3,3,1)
sns.boxplot(x='temp', data=bikesdata)
plt.subplot(3,3,2)
sns.boxplot(x='hum',data=bikesdata)
plt.subplot(3,3,3)
sns.boxplot(x='windspeed',data=bikesdata)

In [499]:

bikesdata = outlier_treatment_columns(bikesdata, num_vars)
bikesdata.shape

## Split to Train and Test Data

In [500]:
from sklearn.model_selection import train_test_split

np.random.seed(0)
bd_train, bd_test = train_test_split(bikesdata, train_size = 0.8, test_size=0.2, random_state=100)

## Rescaling the Features 
I am going to use Min-Max scaling

In [501]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [502]:
bd_train[num_vars] = scaler.fit_transform(bd_train[num_vars])

In [503]:
bd_train.head()

In [504]:
plt.figure(figsize = (40,20))        # Size of the figure
sns.heatmap(bd_train.corr(),annot = True, cmap="YlGnBu")
plt.show()

### Obvervations from above correlation matrix
- Extremely high correlation between temp and atemp, so dropping atemp column
- High correlation between season and month (which makes total sense)

In [505]:
bd_train.drop(['atemp'], axis=1, inplace=True)
#bd_test.drop(['atemp'], axis=1, inplace=True)

### Dividing into X and Y sets for the model building

In [506]:
y_train = bd_train.pop('cnt')
X_train = bd_train

## RFE
- Recursive feature elimination

In [507]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

def calculate_rfe(xtrain, ytrain, num_features):
    lm = LinearRegression()
    lm.fit(xtrain, ytrain)
    
    rfe = RFE(lm, num_features)
    rfe = rfe.fit(xtrain, ytrain)
    
    #list(zip(xtrain.columns,rfe.support_,rfe.ranking_))
    
    return rfe, lm

In [508]:
rfe,_ = calculate_rfe(X_train, y_train, 16)

In [509]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [510]:
cols_include = X_train.columns[rfe.support_]
cols_include

In [511]:
cols_exclude = X_train.columns[~rfe.support_]
cols_exclude

In [512]:
X_train_rfe = X_train[cols_include]

### Build linear regression model

In [513]:
#X_train.columns

In [514]:
# Assign all the feature variables to X
X_train_lm = X_train[['yr', 'holiday', 'workingday', 'temp', 'hum', 'windspeed',
       'season_summer', 'season_fall', 'season_winter', 'mnth_feb', 'mnth_mar',
       'mnth_apr', 'mnth_may', 'mnth_jun', 'mnth_jul', 'mnth_aug', 'mnth_sep',
       'mnth_oct', 'mnth_nov', 'mnth_dec', 'day_tue', 'day_wed', 'day_thu',
       'day_fri', 'day_sat', 'day_sun', 'wthr_cloudy_misty', 'wthr_snow_storms']]

In [515]:
import statsmodels.api as sm

def getLinearRegressionModel(xtrain):
    # add constant
    xtrainlm = sm.add_constant(xtrain)
    
    # build linear model
    lrmodel = sm.OLS(y_train, xtrainlm).fit() 
    lrmodel.summary()
    
    return xtrainlm, lrmodel
    

In [516]:
# build linear model
X_train_lm, lm = getLinearRegressionModel(X_train_rfe)

In [517]:
X_train_rfe.columns

In [518]:
lm.summary()

In [519]:
# drop columns with higher p-value (> 0.05)
def dropInsignificantColumns(lr):    
    numAttributes = len(lr.pvalues)
    drop_columns_index = []
    drop_columns = []

    # pvalues contans const that is not in original column list
    # make a list of all columns which have p-value greater than 0.05
    for attributeIndex in range (1, numAttributes):
        if lr.pvalues[attributeIndex] > 0.05:
            drop_columns_index.append(attributeIndex-1)
            drop_columns.append(X_train.columns[attributeIndex-1])


    print('Drop below columns with p-value greater than 0.05')
    print(drop_columns)
    print(lr.pvalues[drop_columns])
    lr.summary()
    return drop_columns

In [520]:
#drop_columns = dropInsignificantColumns(lm)

## Drop insignificant columns

In [521]:
#day_fri has maximim p-value
X_train_rfe.drop(['day_fri'], axis=1, inplace=True)


In [522]:
_,lm = getLinearRegressionModel(X_train_rfe)
lm.summary()

In [523]:
# day_thu is most insignificant
X_train_rfe.drop(['day_thu'], axis=1, inplace=True)

In [524]:
_,lm = getLinearRegressionModel(X_train_rfe)
lm.summary()

In [525]:
# day_sat is most insignificant
X_train_rfe.drop(['day_sat'], axis=1, inplace=True)

In [526]:
_,lm = getLinearRegressionModel(X_train_rfe)
lm.summary()

In [527]:
# day_wed is most insignificant
X_train_rfe.drop(['day_wed'], axis=1, inplace=True)

In [528]:
_,lm = getLinearRegressionModel(X_train_rfe)
lm.summary()

In [529]:
# day_sat is most insignificant
X_train_rfe.drop(['workingday'], axis=1, inplace=True)

In [530]:
_,lm = getLinearRegressionModel(X_train_rfe)
lm.summary()

In [531]:
# day_tue is most insignificant
X_train_rfe.drop(['day_tue'], axis=1, inplace=True)

In [532]:
_,lm = getLinearRegressionModel(X_train_rfe)
lm.summary()

## Check VIF

In [533]:
from statsmodels.stats.outliers_influence import variance_inflation_factor  

def calculate_vif_recursive(X, thresh=5):
    cols = X.columns
    variables = np.arange(X.shape[1])
    dropped=True
    while dropped:
        dropped=False
        c = X[cols[variables]].values
        vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]
    
        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc))
            variables = np.delete(variables, maxloc)
            dropped=True
        if len(variables) == 1:
            return X[cols[variables]]
            break
    
    print('Remaining variables:')
    print(X.columns[variables])
    return X[cols[variables]]



In [534]:
# Check for the VIF values of the feature variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(xtrain):
    # Create a dataframe that will contain the names of all the feature variables and their respective VIFs
    vif = pd.DataFrame()
    vif['Features'] = xtrain.columns
    vif['VIF'] = [variance_inflation_factor(xtrain.values, i) for i in range(xtrain.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif

In [535]:
#X_train_vif = calculate_vif_recursive(X_train_lm)

In [536]:
vif = calculate_vif(X_train_rfe)
vif

In [537]:
# temp has high vif and dropping that column
X_train_vif = X_train_rfe.drop(['temp'], axis=1)


In [538]:
# Check linear regression again
_,lm = getLinearRegressionModel(X_train_vif)
lm.summary()

#### Observations from dropping "temp" column
- r2 reduces from 0.84 to 0.72
- Not a good idea to drop the temp column
- Lets try dropping column with next highest vif value, which is hum

In [539]:
# hum has high vif and dropping that column
X_train_vif = X_train_rfe.drop(['hum'], axis=1)

In [540]:
# Check linear regression again
_,lm = getLinearRegressionModel(X_train_vif)
lm.summary()

In [541]:
#calculate vif
vif = calculate_vif(X_train_vif)
vif

In [542]:
# dropping temp has high vif and dropping that column
X_train_vif_1 = X_train_vif.drop(['temp'], axis=1)

In [543]:
# Check linear regression again
_,lm = getLinearRegressionModel(X_train_vif_1)
lm.summary()

### Observations from dropping temp column
- r2 greatly reduces from 0.84 to 0.72
- So we keep this column and try dropping season season_fall column instead

In [544]:
# dropping season_fall has high vif and dropping that column
X_train_vif_1 = X_train_vif.drop(['season_fall'], axis=1)

In [545]:
#calculate vif
vif = calculate_vif(X_train_vif_1)
vif

In [546]:
# Check linear regression again
X_train_vif_lm,lm = getLinearRegressionModel(X_train_vif_1)
lm.summary()

In [547]:
X_train = X_train_vif_1
X_train_lm = X_train_vif_lm

### Above linear regression model seems good by following observation
- All the columsn have 0 pvalue
- r2 and r2 adjusted are same and equal to 0.81
- p(f-stat) is almost zero


## Residual Analysis of the train data
So, now to check if the error terms are also normally distributed (which is infact, one of the major assumptions of linear regression), let us plot the histogram of the error terms and see what it looks like.

In [548]:
y_train_cnt = lm.predict(X_train_lm)

In [549]:
# Importing the required libraries for plots.
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [550]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_cnt), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)                         # X-label

#### We can see from histogram above that error terms are normally distributed around zero

### Predictions


In [551]:
X_train.columns

In [552]:
bd_test.columns

In [553]:
#num_vars = ['temp', 'hum', 'windspeed']
bd_test[num_vars] = scaler.transform(bd_test[num_vars])

In [554]:
y_test = bd_test.pop('cnt')
X_test = bd_test

In [555]:
# Now let's use our model to make predictions.

# Creating X_test_new dataframe by dropping variables from X_test
X_test_new = X_test[X_train.columns]

# Adding a constant variable 
X_test_new = sm.add_constant(X_test_new)

In [556]:
# Making predictions
y_pred = lm.predict(X_test_new)

## Model Evaluation

In [557]:
# Plotting y_test and y_pred to understand the spread.
fig = plt.figure()
plt.scatter(y_test,y_pred)
fig.suptitle('y_test vs y_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)                          # Y-label

In [559]:
def calculate_residuals(model, features, label):
    """
    Creates predictions on the features with the model and calculates residuals
    """
    predictions = model.predict(features)
    df_results = pd.DataFrame({'Actual': label, 'Predicted': predictions})
    df_results['Residuals'] = abs(df_results['Actual']) - abs(df_results['Predicted'])
    
    return df_results

In [560]:
bd_results = calculate_residuals(lm, X_test_new, y_test)

In [561]:
# Plotting the actual vs predicted values
sns.lmplot(x='Actual', y='Predicted', data=bd_results, fit_reg=False, height=7)
    
# Plotting the diagonal line
line_coords = np.arange(bd_results.min().min(), bd_results.max().max())
plt.plot(line_coords, line_coords,  # X and y points
          color='darkorange', linestyle='--')
plt.title('Actual vs. Predicted')
plt.show()



The plot shows a linear relationship between the actual and predicted values.

In [564]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

#### r2-score for y-test 0.79, is close to r2-score of y-train (0.81)

## Overall we have decent (tight) linear regression model for BoomBikes rentals